<a href="https://colab.research.google.com/github/getrichthroughcode/Projet-S9/blob/Simon/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Installer matplotlib
!pip install matplotlib

# Installer PyTorch et t+orchvision
!pip install torch torchvision

# Installer PIL (inclus dans Pillow)
!pip install pillow

Charger les Librairies

In [6]:
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import PIL.Image as Image
import glob
from tqdm import tqdm  # Pour afficher une barre de progression
from torch.utils.data import DataLoader
import numpy as np
import zipfile
import os

import csv


def read_csv(path_folder, path_file, list_name):
    csv.field_size_limit(100000000)
    out = [None] * len(list_name)
    path_file = path_folder + '/' + path_file + ".csv"
    with open(path_file, newline='') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        for row in spamreader:
            row = row[0]
            row = row.split(',')
            if row[0] in list_name:
                out[list_name.index(row[0])] = row[1:]
    return out

Charger le CNN

In [7]:

import torch
import torch.nn as nn

class CNN1D(nn.Module):
    def __init__(self, num_classes=2):
        super(CNN1D, self).__init__()

        # Bloc de convolutions
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

        # Pooling et Dropout
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.80)

        # Fully connected layers
        self.fc1 = nn.Linear(16000, 128)  # Ajusté en fonction de la réduction
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        # Ajouter une dimension de canal
        x = x.unsqueeze(1)  # shape (batch_size, 1, 16000)

        # Convolutions et pooling
        x = self.relu(self.conv1(x))
        x = self.pool(x)  # shape (batch_size, 32, 8000)

        x = self.relu(self.conv2(x))
        x = self.pool(x)  # shape (batch_size, 64, 4000)

        x = self.relu(self.conv3(x))
        x = self.pool(x)  # shape (batch_size, 128, 2000)

        # Réduction de la dimension spatiale pour la Fully Connected
        x = x.view(x.size(0), -1)  # Flatten : shape (batch_size, 128 * 1000)
        x = self.dropout(x)

        # Fully connected
        x = self.relu(self.fc1(x))  # shape (batch_size, 128)
        x = self.fc2(x)             # shape (batch_size, num_classes)

        return x


In [8]:
import torch
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size, stride, padding)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.shortcut = nn.Sequential()
        if in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=1),
                nn.BatchNorm1d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        x = self.relu(x)
        return x

class CNN1D(nn.Module):
    def __init__(self, input_length, num_classes=2):
        super(CNN1D, self).__init__()
        self.input_length = input_length

        # Initial convolution
        self.initial_conv = nn.Conv1d(1, 32, kernel_size=5, stride=1, padding=2)
        self.initial_bn = nn.BatchNorm1d(32)
        self.relu = nn.ReLU()

        # Residual blocks
        self.block1 = ResidualBlock(32, 64, kernel_size=5, stride=1, padding=2)
        self.block2 = ResidualBlock(64, 128, kernel_size=3, stride=1, padding=1)
        self.block3 = ResidualBlock(128, 256, kernel_size=3, stride=1, padding=1)

        # Pooling and dropout
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)

        # Calculate flattened size dynamically
        with torch.no_grad():
            dummy_input = torch.zeros(1, 1, self.input_length)
            self.flattened_size = self._get_flattened_size(dummy_input)

        # Fully connected layers
        self.fc1 = nn.Linear(self.flattened_size, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def _get_flattened_size(self, x):
        """Pass a dummy input to calculate flattened size after convolutions."""
        x = self.relu(self.initial_bn(self.initial_conv(x)))
        x = self.pool(x)
        x = self.block1(x)
        x = self.pool(x)
        x = self.block2(x)
        x = self.pool(x)
        x = self.block3(x)
        x = self.pool(x)
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension: (batch_size, 1, input_length)
        x = self.relu(self.initial_bn(self.initial_conv(x)))
        x = self.pool(x)
        x = self.block1(x)
        x = self.pool(x)
        x = self.block2(x)
        x = self.pool(x)
        x = self.block3(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x


Charger le Dataset

In [10]:
zip_path = '/content/Train.zip'  # Remplacez par le chemin réel du fichier ZIP

# Chemin de destination pour extraire les fichiers
extract_path = '/content/Train'

# Extraire tout
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Lister les fichiers extraits
print(os.listdir(extract_path))

['Train']


In [22]:
print(read_csv("test/test","0_MRU",['X']))


class CustomDataset():
    def __init__(self):
        self.imgs_path = "Train/Train/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            data_list = read_csv("Train/Train/", class_path[12:-4], ['type_movement','X']) #TODO ameliorer les fonctions
            class_name = data_list[0][0]
            self.data.append([data_list[1], class_name])
        self.class_map = {"MRU" : 0, "MUA": 1}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #print(self.data)
        position = np.array(self.data[idx][0],dtype='float64')
        class_name = self.data[idx][1]
        class_id = self.class_map[class_name]  # Utilisation correcte du mapping
        #print(f"Item at {idx}:, {class_name} -> Class ID: {class_id}")
        # Convert the tensor to float32
        return torch.from_numpy(position).type(torch.float32), class_id # Changed to torch.float32


dataset = CustomDataset()

print(f"Length of dataset: {len(dataset)}")

dataset.__getitem__(0)

[['0', '-3.81335060346797', '-10.235502204356138', '-10.92359947359968', '-9.292682686332702', '-3.757781409834738', '4.2900135079785375', '12.124878220718152', '16.47155018197715', '23.75009707385834', '31.727959786575585', '39.03240939377007', '47.63914532747418', '53.803089097303406', '60.39309420762143', '66.07373313494145', '71.62911378938851', '76.57684713513088', '82.35837356212745', '87.12010839213877', '88.80019469104343', '87.06116201310628', '84.21451291867261', '83.73898797829226', '85.14498826973325', '87.52371296215324', '88.69028769622757', '86.81292843736938', '83.2141021094363', '78.79481063824812', '74.04285881492234', '71.72834975344354', '69.84053507910525', '71.72386343264868', '76.35178871673294', '80.78825724368608', '85.1730448660321', '89.12504623659355', '93.035772275087', '99.29904036176737', '103.66200777316249', '104.369935845648', '99.94548164410479', '93.01522156028138', '83.55108750444202', '76.69477367834457', '74.06302321004664', '69.63894054462219', '

(tensor([ 0.0000e+00, -1.8612e-01, -6.5353e-01,  ...,  1.8712e+07,
          1.8785e+07,  1.8859e+07]),
 1)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Définir les hyperparamètres
batch_size = 8
learning_rate = 0.001
num_epochs = 10

X = []
Y = []
L = []

# Créer le DataLoader
dataset = CustomDataset()
print(dataset)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers = 2)

# Définir le modèle, la perte, et l'optimiseur
model = CNN1D(1000, num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Boucle d'entraînement
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_idx, (inputs, labels) in enumerate(dataloader):

        # Forward
        outputs = model.forward(inputs)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Statistiques
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Afficher les informations d'epoch
    epoch_loss = running_loss / len(dataloader)
    epoch_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_accuracy:.2f}%")
    Y.append(epoch_accuracy)
    X.append(epoch)
    L.append(epoch_loss)

plt.plot(X, Y)
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Over Epochs')
plt.show()

plt.plot(X, L)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.show()


Epoch 1/10 | Loss: 0.1069 | Accuracy: 98.13%


In [9]:
zip_path = '/content/test.zip'  # Remplacez par le chemin réel du fichier ZIP

# Chemin de destination pour extraire les fichiers
extract_path = '/content/test'

# Extraire tout
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Lister les fichiers extraits
print(os.listdir(extract_path))

['test']


In [ ]:
print(read_csv("test/test","0_MRU",['X']))


class CustomDataset():
    def __init__(self):
        self.imgs_path = "test/test/"
        file_list = glob.glob(self.imgs_path + "*")
        self.data = []
        for class_path in file_list:
            data_list = read_csv("test/test/", class_path[10:-4], ['type_movement','X']) #TODO ameliorer les fonctions
            class_name = data_list[0][0]
            self.data.append([data_list[1], class_name])
        self.class_map = {"MRU" : 0, "MUA": 1}

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        position = np.array(self.data[idx][0],dtype='float64')
        class_name = self.data[idx][1]
        class_id = self.class_map[class_name]  # Utilisation correcte du mapping
        #print(f"Item at {idx}:, {class_name} -> Class ID: {class_id}")
        # Convert the tensor to float32
        return torch.from_numpy(position).type(torch.float32), class_id # Changed to torch.float32




dataset = CustomDataset()

print(f"Length of dataset: {len(dataset)}")

dataset.__getitem__(0)

In [ ]:
# Évaluation sur le jeu de test
def evaluate_model(model, test_loader):
    model.eval()  # Passer en mode évaluation
    correct = 0
    total = 0
    with torch.no_grad():  # Désactiver les gradients
        for inputs, labels in test_loader:
            # Forward
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)  # Obtenir la classe prédite
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    # Calculer l'accuracy
    accuracy = 100 * correct / total
    return accuracy

# Créer le DataLoader pour les données de test
test_dataset = CustomDataset()  # Chargez ici vos données de test
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

# Calculer l'accuracy sur le jeu de test
test_accuracy = evaluate_model(model, test_loader)
print(f"Test Accuracy: {test_accuracy:.2f}%")
